In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense

# Download Apple stock data
aapl = yf.download('AAPL', start='2010-01-01', end='2023-10-31')

[*********************100%%**********************]  1 of 1 completed


In [2]:
# Clean the data
aapl_cleaned = aapl.dropna()

# Perform feature engineering
aapl_features = pd.DataFrame()
aapl_features['Close'] = aapl_cleaned['Close']
aapl_features['MA_50'] = aapl_cleaned['Close'].rolling(window=50).mean()
aapl_features['MA_200'] = aapl_cleaned['Close'].rolling(window=200).mean()

# Normalize the data
aapl_normalized = (aapl_features - aapl_features.min()) / (aapl_features.max() - aapl_features.min())

# Split the data into train and test sets
train_size = int(len(aapl_normalized) * 0.8)
train_data = aapl_normalized[:train_size]
test_data = aapl_normalized[train_size:]

In [3]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense

#train_data_reshaped = train_data.values.reshape((train_data.shape[0], train_data.shape[1], 1, 1))
#test_data_reshaped = test_data.values.reshape((test_data.shape[0], test_data.shape[1], 1, 1))
train_data_reshaped = np.repeat(train_data.values[:, :, np.newaxis], 3, axis=2)
test_data_reshaped = np.repeat(test_data.values[:, :, np.newaxis], 3, axis=2)


In [8]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(train_data.shape[1], train_data.shape[2] , 3))
#base_model = VGG16(weights='imagenet', include_top=False, input_shape=(train_data_reshaped.shape[1], train_data_reshaped.shape[2], 1))
#base_model = VGG16(weights='imagenet', include_top=False, input_shape=(train_data_reshaped.shape[1], train_data_reshaped.shape[2], 3))

IndexError: tuple index out of range

In [9]:
# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Add a custom classifier on top of the base model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

NameError: name 'base_model' is not defined

In [ ]:
# Unfreeze the last few layers of the base model
for layer in model.layers[-5:]:
    layer.trainable = True

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(test_data, test_labels))